In [3]:
import mediapipe as mp # type: ignore
import cv2 # type: ignore
import os
import csv
import numpy as np # type: ignore

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# 1. Make Some Detection

In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

# Loop to continuously capture frames and process them
    while cap.isOpened():
        ret, frame = cap.read()  # Read a frame from the video capture
        
        # Convert the frame color from BGR to RGB for Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame with the holistic model
        results = holistic.process(image)
            
        # Convert the frame color back to BGR for OpenCV
        image.flags.writeable = True
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
        # 1 Draw face landmarks on the frame
        mp_drawing.draw_landmarks(image_bgr, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2, circle_radius=1))
        
        # 2 Draw Right hand landmark on the frame
        mp_drawing.draw_landmarks(image_bgr, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                # BGR Format for the drawingspec color
                                  mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(240,0,0), thickness=2, circle_radius=2))
        
        # 3 Draw Left hand landmark on the frame
        mp_drawing.draw_landmarks(image_bgr, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                #   BGR Frmat for the drawingspec color right hand
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
       
        # 4 Draw Left hand landmark on the frame
        mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # Display the annotated frame
        cv2.imshow('Holistic Model Detection', image_bgr)
            
        # Check for 'q' key press to exit the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

C:\Users\Shruti\AppData\Roaming\Python\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# 2. Capture Landmarks and export to csv

In [6]:
num_coords = len(results.pose_landmarks.landmark)+ len(results.face_landmarks.landmark)
print(num_coords)

501


In [7]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

In [8]:
#Creating Headers for Data Storage:

#When storing the data in a CSV file or a DataFrame, having headers is essential. The headers ('x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', ...) represent each landmark's coordinates and visibility, making the data easy to understand and use.
#This makes the data self-descriptive, meaning anyone reading the CSV file can easily understand what each column represents.

In [11]:
#landmarks

In [8]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [12]:
class_name = "Confident"

In [19]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=0),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=0)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass

        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train a Custom Model Using Scikit Learn

## 3.1 Read in collected data and process

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('coords.csv')

In [3]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.548689,0.433986,-0.817696,0.999567,0.571791,0.368249,-0.772245,0.999181,0.587279,...,-0.004175,0.0,0.615612,0.351858,0.020521,0.0,0.620475,0.345794,0.021175,0.0
1,Happy,0.546373,0.432539,-1.033540,0.999554,0.570429,0.365408,-0.974809,0.999161,0.585743,...,-0.002987,0.0,0.608229,0.348577,0.019040,0.0,0.613252,0.341962,0.019675,0.0
2,Happy,0.544478,0.430101,-1.030677,0.999563,0.569337,0.361225,-0.970915,0.999177,0.584751,...,-0.001765,0.0,0.610060,0.347945,0.019752,0.0,0.615213,0.341849,0.020309,0.0
3,Happy,0.542348,0.430073,-1.064413,0.999527,0.567855,0.361002,-1.009165,0.999126,0.583461,...,-0.001303,0.0,0.605841,0.349637,0.019445,0.0,0.610994,0.343642,0.020023,0.0
4,Happy,0.541618,0.429715,-1.076290,0.999505,0.567376,0.360684,-1.022262,0.999086,0.582947,...,-0.003950,0.0,0.607839,0.348010,0.016190,0.0,0.612873,0.342235,0.016628,0.0


In [4]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
960,Confident,0.463282,0.470706,-1.162110,0.999874,0.490572,0.403339,-1.135808,0.999736,0.511648,...,0.002556,0.0,0.546994,0.359503,0.028247,0.0,0.552503,0.352105,0.029200,0.0
961,Confident,0.468164,0.454106,-1.055598,0.999837,0.493885,0.388348,-1.026606,0.999655,0.514104,...,0.002368,0.0,0.547083,0.359507,0.028353,0.0,0.552470,0.352056,0.029343,0.0
962,Confident,0.469970,0.445518,-1.030721,0.999819,0.496388,0.377071,-0.997978,0.999618,0.515970,...,0.001251,0.0,0.541311,0.352857,0.022672,0.0,0.547208,0.346090,0.023385,0.0
963,Confident,0.469956,0.438677,-1.007399,0.999777,0.495208,0.370244,-0.971881,0.999543,0.514104,...,0.001882,0.0,0.536938,0.354257,0.023453,0.0,0.542725,0.347940,0.024106,0.0
964,Confident,0.469627,0.436283,-0.964761,0.999750,0.494898,0.367341,-0.928238,0.999503,0.513439,...,0.002231,0.0,0.536188,0.353267,0.023399,0.0,0.542095,0.346653,0.024090,0.0


In [ ]:
# df(df['class]=='')

In [ ]:
# CORRDINATED X,YZ ARE THE FEATURES AND CLASS IS THE TARGET. Eg Confident is the target and x,y,z corrdinates are the features.

In [9]:
x = df.drop('class', axis = 1)

In [10]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.548689,0.433986,-0.817696,0.999567,0.571791,0.368249,-0.772245,0.999181,0.587279,0.368541,...,-0.004175,0.0,0.615612,0.351858,0.020521,0.0,0.620475,0.345794,0.021175,0.0
1,0.546373,0.432539,-1.033540,0.999554,0.570429,0.365408,-0.974809,0.999161,0.585743,0.365864,...,-0.002987,0.0,0.608229,0.348577,0.019040,0.0,0.613252,0.341962,0.019675,0.0
2,0.544478,0.430101,-1.030677,0.999563,0.569337,0.361225,-0.970915,0.999177,0.584751,0.361507,...,-0.001765,0.0,0.610060,0.347945,0.019752,0.0,0.615213,0.341849,0.020309,0.0
3,0.542348,0.430073,-1.064413,0.999527,0.567855,0.361002,-1.009165,0.999126,0.583461,0.361168,...,-0.001303,0.0,0.605841,0.349637,0.019445,0.0,0.610994,0.343642,0.020023,0.0
4,0.541618,0.429715,-1.076290,0.999505,0.567376,0.360684,-1.022262,0.999086,0.582947,0.360834,...,-0.003950,0.0,0.607839,0.348010,0.016190,0.0,0.612873,0.342235,0.016628,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,0.463282,0.470706,-1.162110,0.999874,0.490572,0.403339,-1.135808,0.999736,0.511648,0.403315,...,0.002556,0.0,0.546994,0.359503,0.028247,0.0,0.552503,0.352105,0.029200,0.0
961,0.468164,0.454106,-1.055598,0.999837,0.493885,0.388348,-1.026606,0.999655,0.514104,0.389348,...,0.002368,0.0,0.547083,0.359507,0.028353,0.0,0.552470,0.352056,0.029343,0.0
962,0.469970,0.445518,-1.030721,0.999819,0.496388,0.377071,-0.997978,0.999618,0.515970,0.378185,...,0.001251,0.0,0.541311,0.352857,0.022672,0.0,0.547208,0.346090,0.023385,0.0
963,0.469956,0.438677,-1.007399,0.999777,0.495208,0.370244,-0.971881,0.999543,0.514104,0.370974,...,0.001882,0.0,0.536938,0.354257,0.023453,0.0,0.542725,0.347940,0.024106,0.0


In [11]:
y = df['class']
y

0          Happy
1          Happy
2          Happy
3          Happy
4          Happy
         ...    
960    Confident
961    Confident
962    Confident
963    Confident
964    Confident
Name: class, Length: 965, dtype: object

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [13]:
y_test

446    Confident
382    Confident
501    Confident
230        Happy
739    Confident
         ...    
876    Confident
170        Happy
807    Confident
636    Confident
950    Confident
Name: class, Length: 290, dtype: object

## 3.2 Train Machine learning Classification Model

In [14]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [16]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [19]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [21]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

C:\Users\Shruti\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
fit_models


{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [24]:
fit_models['rc'].predict(x_test)

array(['Confident', 'Confident', 'Confident', 'Happy', 'Confident',
       'Happy', 'Confident', 'Confident', 'Confident', 'Confident',
       'Happy', 'Confident', 'Confident', 'Confident', 'Confident',
       'Confident', 'Confident', 'Happy', 'Happy', 'Happy', 'Confident',
       'Confident', 'Confident', 'Happy', 'Confident', 'Happy',
       'Confident', 'Confident', 'Confident', 'Happy', 'Confident',
       'Confident', 'Confident', 'Confident', 'Happy', 'Happy',
       'Confident', 'Confident', 'Happy', 'Happy', 'Confident',
       'Confident', 'Happy', 'Confident', 'Confident', 'Confident',
       'Confident', 'Confident', 'Happy', 'Confident', 'Confident',
       'Confident', 'Confident', 'Confident', 'Confident', 'Happy',
       'Confident', 'Confident', 'Confident', 'Confident', 'Confident',
       'Confident', 'Confident', 'Confident', 'Happy', 'Happy',
       'Confident', 'Happy', 'Confident', 'Confident', 'Confident',
       'Confident', 'Confident', 'Confident', 'Confiden

## 3.3 Evaluate and Serialize Model